<a href="https://colab.research.google.com/github/Bayzid03/LangGraph-Hub/blob/main/Weekend_Activity_Planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Still Working on this project

In [ ]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image
from dotenv import load_dotenv
import random

In [ ]:
# Load environment variables and model
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

In [ ]:
# Define Agent state
class WeekendPlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
    location: str
    preferences: List[str]
    weather_info: str
    weekend_plan: str

# Prompt templates
weather_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a weather assistant. Provide a realistic weekend weather forecast for {location}. Include Saturday and Sunday conditions, temperature, and any weather advisories."),
    ("human", "What's the weather forecast for this weekend in {location}?"),
])

weekend_plan_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful weekend activity planner. Create a comprehensive 2-day weekend plan for {location} based on the user's preferences: {preferences} and current weather conditions: {weather_info}.

Structure your response as:
**SATURDAY**
- Morning (9 AM - 12 PM): [Activity]
- Afternoon (12 PM - 5 PM): [Activity]
- Evening (5 PM - 9 PM): [Activity]

**SUNDAY**
- Morning (9 AM - 12 PM): [Activity]
- Afternoon (12 PM - 5 PM): [Activity]
- Evening (5 PM - 9 PM): [Activity]

Consider weather conditions and provide specific locations, addresses when possible, and brief descriptions of each activity."""),
    ("human", "Create a weekend activity plan for me."),
])

In [ ]:
# Defining Agent Functions
def input_location(state: WeekendPlannerState) -> WeekendPlannerState:
    print("Please enter your city/location for the weekend activities:")
    user_message = input("Your input: ")
    return {
        **state,
        "location": user_message,
        "messages": state['messages'] + [HumanMessage(content=user_message)],
    }

def input_preferences(state: WeekendPlannerState) -> WeekendPlannerState:
    print(f"Please enter your activity preferences for {state['location']} (comma-separated):")
    print("Examples: outdoor activities, museums, restaurants, shopping, nightlife, cultural sites, adventure sports, relaxing activities")
    user_message = input("Your input: ")
    return {
        **state,
        "preferences": [preference.strip() for preference in user_message.split(',')],
        "messages": state['messages'] + [HumanMessage(content=user_message)],
    }

def check_weather(state: WeekendPlannerState) -> WeekendPlannerState:
    print(f"Checking weather forecast for {state['location']}...")

    # Get weather forecast using LLM
    response = llm.invoke(weather_prompt.format_messages(location=state['location']))
    weather_info = response.content

    print(f"\nWeather Forecast:")
    print(weather_info)
    print("\n" + "="*50 + "\n")

    return {
        **state,
        "weather_info": weather_info,
        "messages": state['messages'] + [AIMessage(content=weather_info)],
    }

def create_weekend_plan(state: WeekendPlannerState) -> WeekendPlannerState:
    print(f"Creating weekend activity plan for {state['location']}...")
    print(f"Based on preferences: {', '.join(state['preferences'])}")
    print("Considering weather conditions...\n")

    response = llm.invoke(weekend_plan_prompt.format_messages(
        location=state['location'],
        preferences=", ".join(state['preferences']),
        weather_info=state['weather_info']
    ))

    print("🎉 Your Weekend Activity Plan:")
    print("="*50)
    print(response.content)
    print("="*50)

    return {
        **state,
        "messages": state['messages'] + [AIMessage(content=response.content)],
        "weekend_plan": response.content,
    }